In [7]:
import csv
import json
import os
import requests
from bs4 import BeautifulSoup

In [5]:
url = "https://s.cafef.vn/screener.aspx"

In [3]:
def get_html_data():
    response = requests.get(url)
    return response.text

In [15]:
def crawl_data():
    html_data = get_html_data()
    soup = BeautifulSoup(html_data, "html.parser")

    # Extract all script tags
    script_tags = soup.find_all("script")

    # Iterate over the script tags and print their content or attributes
    for script_tag in script_tags:
        # If you want to print the content inside the script tag
        if "var jsonData =" in script_tag.text:
            list_json_data = (
                script_tag.text.split("var jsonData =")[1]
                .split(";")[0]
                .split("[")[1]
                .split("]")[0]
                .split("},")
            )

            json_data = []
            for idx, item in enumerate(list_json_data):
                if idx == len(list_json_data) - 1:
                    json_data.append(json.loads(item.replace("(-?)Infinity|NaN", "null")))
                else:
                    json_data.append(json.loads(item.replace("(-?)Infinity|NaN", "null") + "}"))
            update_data_in_csv_file( json_data)
                


def update_data_in_csv_file( datas):
    script_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else './data/'
    file_path = os.path.join(script_dir, f"data.csv")

    # Use 'w' mode to create the file or overwrite existing content
    with open(file_path, "w", newline="", encoding="utf-8-sig") as csvfile:
        fieldnames = datas[0].keys() if datas else []
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write header
        writer.writeheader()

        # Write data rows
        writer.writerows(datas)

    print(f"Data written to CSV file")

if __name__ == "__main__":
    crawl_data()

Data written to CSV file
